# TP DFA dans RSA-CRT

## 1/Prise en main de la librarie crypto Python PyCryptodome

In [6]:
#output est le résultat de la signature du message "Hello World!" par le protocole PKCS#1v1.5 utilisant la fonction de hachage 
#SHA-256
output = "r4F09799F6A59081B725599753330B7A2440ABC42606601622FE0C582646E32555303E1062A2989D9B4C265431ADB58DD\nz00\nr85BB33C4BB237A311BC40C1279528FD6BB36F94F534A4D8284A18AB8E5670E734C55A6CCAB5FB5EAE02BA37E2D56648D\nz00\nr7A13BBF17A0E07D607C07CBB72C7A7A77076376E8434CE6E136832DC95DB3D80\nz00"
print(output)

r4F09799F6A59081B725599753330B7A2440ABC42606601622FE0C582646E32555303E1062A2989D9B4C265431ADB58DD
z00
r85BB33C4BB237A311BC40C1279528FD6BB36F94F534A4D8284A18AB8E5670E734C55A6CCAB5FB5EAE02BA37E2D56648D
z00
r7A13BBF17A0E07D607C07CBB72C7A7A77076376E8434CE6E136832DC95DB3D80
z00


In [7]:
#On affiche le résultat en hexadécimal
newout = output.replace("r", "").replace("\nz00","").replace("\n","")
print(newout)

4F09799F6A59081B725599753330B7A2440ABC42606601622FE0C582646E32555303E1062A2989D9B4C265431ADB58DD85BB33C4BB237A311BC40C1279528FD6BB36F94F534A4D8284A18AB8E5670E734C55A6CCAB5FB5EAE02BA37E2D56648D7A13BBF17A0E07D607C07CBB72C7A7A77076376E8434CE6E136832DC95DB3D80


In [8]:
#Conversion en binaire avec binascii
from binascii import unhexlify, hexlify
sig = unhexlify(newout)
print(sig)

b'O\ty\x9fjY\x08\x1brU\x99u30\xb7\xa2D\n\xbcB`f\x01b/\xe0\xc5\x82dn2US\x03\xe1\x06*)\x89\xd9\xb4\xc2eC\x1a\xdbX\xdd\x85\xbb3\xc4\xbb#z1\x1b\xc4\x0c\x12yR\x8f\xd6\xbb6\xf9OSJM\x82\x84\xa1\x8a\xb8\xe5g\x0esLU\xa6\xcc\xab_\xb5\xea\xe0+\xa3~-Vd\x8dz\x13\xbb\xf1z\x0e\x07\xd6\x07\xc0|\xbbr\xc7\xa7\xa7pv7n\x844\xcen\x13h2\xdc\x95\xdb=\x80'


In [9]:
#Chargement des fonctions cryptographiques nécessaires de PyCryptodome
#Chargement du RSA :
from Crypto.PublicKey import RSA
#/!\Ajouter PKCS_v1_5 et SHA256

#Clé publique 
e = 0x10001
N = 0x9292758453063D803DD603D5E777D7888ED1D5BF35786190FA2F23EBC0848AEADDA92CA6C3D80B32C4D109BE0F36D6AE7130B9CED7ACDF54CFC7555AC14EEBAB93A89813FBF3C4F8066D2D800F7C38A81AE31942917403FF4946B0A83D3D3E05EE57C6F5F5606FB5D4BC6CD34EE0801A5E94BB77B07507233A0BC7BAC8F90F79

#Message
m = b"Hello World!"

#Hashage du message avec SHA-256 et affichage du hashé (nommé "hash_object")

#Construction de la clé publique RSA, et affichage en format PEM (les formats DER et OpenSSH sont aussi disponibles) 

#Vérification de la signature PKCS v1.5 du message

#Afficher "True" si la signature est vérifiée (utiliser assert)

## 2/Simulation des fautes dans le RSA-CRT

### 2/1/Calcul du RSA-CRT

In [11]:
# Padding du PKCS#1 v1.5
def build_message(m, N):
    sha_id = "3031300d060960864801650304020105000420"
    N_len = (len(bin(N)) - 2 + 7) // 8
    pad_len = (len(hex(N)) - 2) // 2 - 3 - len(m)//2 - len(sha_id)//2
    padded_m = "0001" + "ff" * pad_len + "00" + sha_id + m
    return padded_m

print("Message:       {}".format(m))

# Encode message
hash_object = SHA256.new(data=m)
hashed_m = hexlify(hash_object.digest()).decode()
padded_m = build_message(hashed_m, N)
msg = int.from_bytes(unhexlify(padded_m), byteorder='big') 
print("Padded/hashed: {}".format(padded_m))

Message:       b'Hello World!'


NameError: name 'SHA256' is not defined

In [13]:
# Signature RSA-CRT
from gmpy2 import invert, powmod, gcd, gcdext

# Paramètres RSA
p = 0xc36d0eb7fcd285223cfb5aaba5bda3d82c01cad19ea484a87ea4377637e75500fcb2005c5c7dd6ec4ac023cda285d796c3d9e75e1efc42488bb4f1d13ac30a57
q = 0xc000df51a7c77ae8d7c7370c1ff55b69e211c2b9e5db1ed0bf61d0d9899620f4910e4168387e3c30aa1e00c339a795088452dd96a9a5ea5d9dca68da636032af

#Calcul de phi et de d

#Paramètres CRT : calculs de dp, dq, qinv

#Calculs internes du CRT : sp, sq et la signature finale (notée s_crt)

#Utiliser la signature précédente correcte, et vérifier que les deux signatures RSA-CRT et non-CRT sont égales
s = int.from_bytes(sig, byteorder='big') 
print("Signature:  {}".format(hex(s)))
print("s == s_crt? {}".format(s == s_crt))

Signature:  0x4f09799f6a59081b725599753330b7a2440abc42606601622fe0c582646e32555303e1062a2989d9b4c265431adb58dd85bb33c4bb237a311bc40c1279528fd6bb36f94f534a4d8284a18ab8e5670e734c55a6ccab5fb5eae02ba37e2d56648d7a13bbf17a0e07d607c07cbb72c7a7a77076376e8434ce6e136832dc95db3d80


NameError: name 's_crt' is not defined

### 2/2/Injection de fautes

In [14]:
# Injection de fautes (bit flip) aléatoires dans sp
from random import randint, sample

# Inversion arbitraire de bits dans sp, et calcul de la signature corrompue

### 2/3/Exploitation des fautes

In [15]:
# Variante de l'attaque de Bellcore n°1 : retrouver p et q à partir de la signature correcte et fautée (DFA)

In [16]:
# Variante de l'attaque de Bellcore n°2 : retrouver p et q à partir de seulement une signature fautée (SFA)

In [17]:
# Calcul de d

In [18]:
# Maintenant que l'on a retrouvé tous les paramètres, on peut déchiffrer le message

# Questions additionnelles :


## 1/Si on faute sq, est-ce que l'attaque réussit ?
## 2/Si on faute iq, est-ce que l'attaque réussit ?
## 3/Ecrire le schéma de protection "BOS Algorithm", et tenter de reproduire les fautes qui contournent cette protection. Idem pour la variante "BOS+".
## 4/Même question pour le schéma de Vigilant